In [2]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_27848/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [4]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
75,model_30_7_10,0.988412,0.804699,0.981097,0.966296,0.977518,0.077488,1.305981,0.219178,0.147212,0.183195,0.503922,0.278367,1.011124,0.290217,103.115264,162.840180,"Hidden Size=[12], regularizer=0.2, learning_ra..."
76,model_30_7_9,0.988396,0.803299,0.982553,0.968587,0.979168,0.077593,1.315337,0.202296,0.137205,0.169750,0.526308,0.278555,1.011139,0.290414,103.112559,162.837475,"Hidden Size=[12], regularizer=0.2, learning_ra..."
77,model_30_7_11,0.988338,0.805871,0.979728,0.964110,0.975958,0.077983,1.298140,0.235055,0.156762,0.195908,0.483683,0.279254,1.011195,0.291143,103.102527,162.827443,"Hidden Size=[12], regularizer=0.2, learning_ra..."
78,model_30_7_8,0.988260,0.801626,0.984098,0.970958,0.980902,0.078506,1.326527,0.184385,0.126851,0.155618,0.551029,0.280189,1.011270,0.292117,103.089163,162.814079,"Hidden Size=[12], regularizer=0.2, learning_ra..."
79,model_30_7_12,0.988199,0.806856,0.978443,0.962044,0.974490,0.078913,1.291553,0.249950,0.165786,0.207868,0.465406,0.280915,1.011329,0.292874,103.078819,162.803734,"Hidden Size=[12], regularizer=0.2, learning_ra..."
80,model_30_7_13,0.988014,0.807686,0.977242,0.960107,0.973116,0.080147,1.286007,0.263884,0.174246,0.219065,0.448909,0.283103,1.011506,0.295155,103.047783,162.772698,"Hidden Size=[12], regularizer=0.2, learning_ra..."
81,model_30_7_7,0.987963,0.799620,0.985734,0.973369,0.982713,0.080488,1.339942,0.165412,0.116318,0.140865,0.578278,0.283705,1.011555,0.295783,103.039285,162.764201,"Hidden Size=[12], regularizer=0.2, learning_ra..."
84,model_30_7_14,0.987800,0.808386,0.976121,0.958303,0.971835,0.081583,1.281326,0.276878,0.182127,0.229502,0.434031,0.285627,1.011712,0.297787,103.012270,162.737185,"Hidden Size=[12], regularizer=0.2, learning_ra..."
85,model_30_7_15,0.987567,0.808978,0.975079,0.956630,0.970645,0.083142,1.277367,0.288962,0.189434,0.239198,0.420625,0.288344,1.011936,0.300619,102.974410,162.699325,"Hidden Size=[12], regularizer=0.2, learning_ra..."
86,model_24_8_7,0.987463,0.808469,0.980767,0.941044,0.970568,0.083835,1.280769,0.167230,0.218856,0.193043,0.439689,0.289543,1.017699,0.301870,86.957800,136.931709,"Hidden Size=[10], regularizer=0.05, learning_r..."
